In [ ]:
# Tests for base Horizon functionality: initialization, dumping, visualizations, etc.
import os
import sys
import warnings
import numpy as np

warnings.filterwarnings('ignore')

sys.path.insert(0, '../../../seismiqb')

from seismiqb import Field, Horizon

In [ ]:
""" You can manage cube and horizon for the test:

CUBE_PATH : str
    Path to an existed seismic cube.
HORIZON_PATH : str
    Path to an existed seismic horizon.
"""
# Tests parameters
OUTPUT_DIR = './horizon_test_files'

CUBE_PATH = os.path.join(OUTPUT_DIR, 'test_cube.sgy')
HORIZON_PATH = os.path.join(OUTPUT_DIR, 'test_horizon')

# Visualization parameters
SCALE = 1
SHOW_FIGURES = True

# Initialization

In [ ]:
%%time
field = Field(CUBE_PATH)
horizon = Horizon(HORIZON_PATH, field=field)
horizon.filter()

horizon.show(show=SHOW_FIGURES, scale=SCALE)

In [ ]:
%%time
# from_points init
new_horizon = Horizon(storage=horizon.points, field=horizon.field, name='tester')

assert horizon.equal(new_horizon), f"`from_points` initialization test failed: original and initialized horizons are unequal"

In [ ]:
%%time
# from_matrix init
new_horizon = Horizon(storage=horizon.matrix, i_min=horizon.i_min, x_min=horizon.x_min,
                      field=horizon.field, name='tester')

assert horizon.equal(new_horizon), f"`from_matrix` initialization test failed: original and initialized horizons are unequal"

In [ ]:
%%time
# from_full_matrix init
new_horizon = Horizon(storage=horizon.full_matrix, field=horizon.field, name='tester')

assert horizon.equal(new_horizon), f"`from_full_matrix` initialization test failed: original and initialized horizons are unequal"

In [ ]:
%%time
# from_dict
horizon_dict = {}
for k, v in zip(horizon.points[:, :2], horizon.points[:, 2]):
    horizon_dict[tuple(k)] = v

new_horizon = Horizon(storage=horizon_dict, transform=False, field=horizon.field, name='tester')

assert horizon.equal(new_horizon), f"`from_dict` initialization test failed: original and initialized horizons are unequal"

# Dump

In [ ]:
dump_path = os.path.join(OUTPUT_DIR, 'tmp_horizon')

In [ ]:
%%time
# dump
horizon.dump(path=dump_path)

dumped_horizon = Horizon(storage=dump_path, field=field)

assert horizon.equal(new_horizon), f"`dump` test failed: original and dumped horizons are unequal"

os.remove(dump_path)

In [ ]:
%%time
# dump_float
max_depth_difference = 2
horizon.dump_float(path=dump_path, max_depth_difference=max_depth_difference)

dumped_horizon = Horizon(storage=dump_path, field=field)

error_message = "`dump_float` test failed: original and dumped horizons are not close"
assert np.allclose(horizon.points, dumped_horizon.points, atol=max_depth_difference), error_message

os.remove(dump_path)

# Base methods

In [ ]:
%%time
horizon_points = Horizon.matrix_to_points(matrix=horizon.matrix)

# Shift (it exists in methods that call `matrix_to_points`)
horizon_points[:, 0] += horizon.i_min
horizon_points[:, 1] += horizon.x_min

error_message = "`matrix_to_points` test failed: original and extracted points are not equal"
assert np.array_equal(horizon.points, horizon_points), error_message

In [ ]:
%%time
horizon_matrix = Horizon.points_to_matrix(points=horizon.points, i_min=horizon.i_min, x_min=horizon.x_min,
                                          i_length=horizon.i_length, x_length=horizon.x_length)

error_message = "`points_to_matrix` test failed: original and extracted matrices are not equal"
assert np.array_equal(horizon.matrix, horizon_matrix), error_message

# Evaluate

In [ ]:
%%time
_ = horizon.evaluate(scale=SCALE, bad_color='black')

# Get values

In [ ]:
%%time
for axis in range(2):
    index = horizon.shape[axis] // 2
    _ = horizon.load_slide(index=index, axis=axis)

    horizon.show_slide(index=index, axis=axis, width=5, combine='separate',
                       cmap=['gray', 'viridis'], grid=False,
                       scale=SCALE, show=SHOW_FIGURES)

# get_cube_values

In [ ]:
%%time
# Make a flat horizon in the cube
depth = field.shape[-1] // 2
window = 2 * (field.shape[-1] // 40) + 1 # must be an odd value

constant_matrix = np.ones_like(horizon.full_matrix) * depth

constant_horizon = Horizon(constant_matrix, i_min=0, x_min=0,
                           field=horizon.field, name='const')

# Get values along constant horizon
horizon_values = constant_horizon.get_cube_values(window=window)
horizon_values = np.nan_to_num(horizon_values, constant_horizon.FILL_VALUE)

# Get values from the cube
geometry_values = field.geometry[:, :, depth-window//2:depth+window//2+1]

error_message = "`get_cube_values` test failed: original and extracted values are not equal"
assert np.allclose(horizon_values[constant_horizon.mask],
                   geometry_values[constant_horizon.mask]), error_message

# Operations with FILL_VALUE

In [ ]:
%%time
# Change fill_value to a new one
horizon_copy = horizon.copy()
new_fill_value = -10

# New and old fill_values amount before filling
n_fill_value = np.sum(horizon_copy.matrix == new_fill_value)
n_absent = np.sum(horizon_copy.matrix == horizon_copy.FILL_VALUE)

horizon_copy.matrix_fill_to_num(matrix=horizon_copy.matrix, value=new_fill_value)

# New fill_values amount after filling
n_new_fill_value = np.sum(horizon_copy.matrix == new_fill_value)

error_message = "`matrix_fill_to_num` test failed: unexpected amount of points with new fill value"
assert n_new_fill_value == n_fill_value + n_absent, error_message

In [ ]:
%%time
# Reverse filling: new fill_value to old
horizon_copy.matrix_num_to_fill(matrix=horizon_copy.matrix, value=new_fill_value)

n_restored_fill_value = np.sum(horizon_copy.matrix == horizon_copy.FILL_VALUE)

error_message = "`matrix_num_to_fill` test failed: unexpected amount of points with restored fill value"
assert n_restored_fill_value == n_new_fill_value, error_message

# Matrix normalization

In [ ]:
%%time
# Min-max
horizon_copy = horizon.copy()

normalized_matrix = horizon_copy.matrix_normalize(matrix=horizon_copy.full_matrix, mode='min-max')

error_msg = "'min-max' matrix normalization test failed: matrix values are not in [0, 1] interval"
assert np.isclose(np.nanmin(normalized_matrix[horizon_copy.mask]), 0), error_msg
assert np.isclose(np.nanmax(normalized_matrix[horizon_copy.mask]), 1), error_msg

# Mean-std
horizon_copy = horizon.copy()

normalized_matrix = horizon_copy.matrix_normalize(matrix=horizon_copy.full_matrix, mode='mean-std')

error_msg = "'mean-std' matrix normalization test failed: matrix values are not in [0, 1] interval"
assert np.isclose(np.nanmean(normalized_matrix[horizon_copy.mask]), 0), error_msg
assert np.isclose(np.nanstd(normalized_matrix[horizon_copy.mask]), 1), error_msg